# Training an agent to play Super Mario using player recorded data

In this exercise you will learn how to use player generated data to train a neural network to play Super Mario. The results will be evaluated against the results from the Q_Learner exercise. 

## 1. Generating data
First, you will have to generate some data for the neural network to train with.


### 1.1 Player generated data.

![Mario](https://media1.giphy.com/media/aX0RqLt2ARSW4/giphy.gif?cid=ecf05e47fnkts3fqh25tj9v8noh9vnccwo4x0ey4zpdxc7ft&rid=giphy.gif&ct=g)


To achieve the best possible results, the training algorithm needs the best possible data. In this case that means player generated data.


You will have the most fun playing with a USB-Controller but if you have none, you can set the following variable to ``False`` to use the keyboard:

In [ ]:
# Don't forget to run me
USE_GAMEPAD = False

### Controls
|            	| Keyboard    	| Xbox       	| Playstation 	|
|------------	|-------------	|------------	|-------------	|
| Jump       	| S           	| A          	| X           	|
| Sprint     	| A           	| B          	| O           	|
| Move Right 	| Arrow Right 	| Dpad Right 	| Dpad Right  	|
| Move Left  	| Arrow Left  	| Dpad Left  	| Dpad Left   	|
| Duck       	| Arrow Down  	| Dpad Down  	| Dpad Down   	|

### Level
The level we will be using for this exercise will be a very easy one to minimize the training time.

However, if you would like to try different levels, we encourage you to do so by changing the ``level`` variable below to a different one from the ``exercise_offline_rl\levels`` folder.

In [ ]:
# Setup the imports and global variables. Run this cell again if you encounter any import errors.
import copy
import d3rlpy
import numpy as np
import os
import pathlib

from d3rlpy.dataset import MDPDataset
from d3rlpy.metrics.scorer import evaluate_on_environment
from sklearn.model_selection import train_test_split

from gym_setup import Env
from Controller.gamepad_controller import GamepadController
from Controller.keyboard_controller import KeyboardController
from data.datasets.getDatasets import getDataset
%load_ext tensorboard

level_name = "OneCliffLevel.lvl"
# ^ change here if you want to try a different level.

level = os.path.join("levels", level_name)

dataset_path = os.path.join(
    "data", "datasets", os.path.split(level)[1] + ".h5")

dataset_path_rand = os.path.join(
    "data", "datasets", os.path.split(level)[1] + ".random.h5")

MODEL_DIR = pathlib.Path("data", "models")
if not MODEL_DIR.exists():
    MODEL_DIR.mkdir(parents=True)


print("Done!")


To start the game run the next cell. If you think you have enough data just close the game window and move on to the next cell.

Note: We have pregenerated some training data for your convenience which will be used in addition to your data to train the model.
If you want to train with your own data only, go ahead and delete the data from ``exercise_offline_rl\data\datasets``.

In [ ]:
# Let's play!
try:
    env_play = Env(visible=True, level=level).env
    if USE_GAMEPAD:
        controller = GamepadController(env_play)
    else:
        controller = KeyboardController(env_play)
    while True:
        observation = env_play.reset()
        done = False
        action = controller.read()

        observations = [observation]
        actions = [action]
        # No reward at first time step, because no action was taken yet
        rewards = [0]
        terminals = [done]

        while not done:
            observation, reward, done, info = env_play.step(action)
            action = controller.read()

            observations.append(observation)
            actions.append(action)
            rewards.append(reward)
            terminals.append(done)

        if os.path.isfile(dataset_path):
            dataset = MDPDataset.load(dataset_path)
            dataset.append(np.asarray(observations), np.asarray(actions), np.asarray(rewards),
                            np.asarray(terminals))
        else:
            dataset = MDPDataset(np.asarray(observations), np.asarray(actions), np.asarray(rewards),
                                    np.asarray(terminals), discrete_action=True)
        dataset.dump(dataset_path)
        stats = dataset.compute_stats()
        mean = stats['return']['mean']
        std = stats['return']['std']
        print(f'mean: {mean}, std: {std}')
except ConnectionResetError:
    print("Done")


### 1.2 Randomly generated data (optional)
To complement the player generated data, it is possible to also generate some random data for the algorithm to train with.

In [ ]:
# Generate random data

EPISODES = 2 # <--- increase if you want more random data. More data might slow down the training process.

env_rand = Env(visible=False, level=level).env

observations = []
actions = []
rewards = []
terminals = []

for episode in range(0, EPISODES):
    observation = env_rand.reset()
    done = False

    while not done:
        action = env_rand.action_space.sample()
        observation, reward, done, info = env_rand.step(action)
        observations.append(observation)
        actions.append(action)
        rewards.append(reward)
        terminals.append(done)

    if episode != 0:
        if os.path.isfile(dataset_path_rand):
            dataset = MDPDataset.load(dataset_path_rand)
            dataset.append(np.asarray(observations), np.asarray(actions), np.asarray(rewards),
                           np.asarray(terminals))
        else:
            dataset = MDPDataset(np.asarray(observations), np.asarray(actions), np.asarray(rewards),
                                 np.asarray(terminals), discrete_action=True)
        dataset.dump(dataset_path_rand)
        stats = dataset.compute_stats()
        mean = stats['return']['mean']
        std = stats['return']['std']
        print(f'mean: {mean}, std: {std}')

print("Done!")


## 2. Use the generated data to train a policy
Now that you have generated some data for the neural network to train with, let's begin with the training.
For the purpose of this exercise we will use the Offline RL Python library [d3rlpy](https://github.com/takuseno/d3rlpy).



### 2.1 Choosing an algorithm
![DQN](https://raw.githubusercontent.com/koerners/marioai/master/exercise_offline_rl/data/jupyter/dqn.PNG)

#### Why we chose DQN
The Deep Q-Network approach is known to have been able to achieve human-level control in Atari games, and it is able to learn successful policies directly from high-dimensional sensory inputs (like pixels) using end-to-end reinforcement learning which makes it ideal for our purpose. [[1]](https://www.nature.com/articles/nature14236) 

It uses data collected from an environment to learn and train without interacting with it.

For more information on DQN, please refer to [this paper](https://www.nature.com/articles/nature14236).



### 2.2 Setup the training
Let's setup some parameters before the training:

In [ ]:
# Training parameters
gamma = 0.99 # discount facor, how important future rewards are
learning_rate = 0.0003 # to what extent the agent overrides old information with new information
target_update_interval = 3000 # intervall of steps that the agent uses to update target network
n_epochs = 100 # <--- change here if you want to train more / less
test_size = 0.1
batch_size = 2 # size of training examples utilized in one iteration
n_frames = 1 # number of frames stacked for image observation
n_steps = 40 # number of steps for td calculation
use_gpu = False # usage of gpu to train

print("Done!")


### 2.3 Training time!

If you want to track the training with tensorboard, run the following cell.

In [ ]:
# Start tensorboard 
%tensorboard --logdir runs

To start the training run the next cell: 

In [ ]:
env_train = Env(visible=False, level=level, run_server=True).env

dataset = getDataset()

train_episodes, test_episodes = train_test_split(dataset, test_size=test_size)

dqn = d3rlpy.algos.DQN(learning_rate=learning_rate, gamma=gamma, use_gpu=use_gpu,
                        target_update_interval=target_update_interval, batch_size=batch_size)

# train offline
dqn.build_with_dataset(dataset)
# set environment in scorer function
evaluate_scorer = evaluate_on_environment(env_train)
# evaluate algorithm on the environment
rewards = evaluate_scorer(dqn)
name = 'marioai_%s_%s_%s_%s_%s' % (level_name, gamma, learning_rate, target_update_interval, n_epochs)
model_file = pathlib.Path(MODEL_DIR, name + ".pt")
currentMax = -100000
dqn_max = copy.deepcopy(dqn)

for epoch, metrics in (dqn.fitter(train_episodes, eval_episodes=test_episodes, tensorboard_dir='runs', experiment_name=name, n_epochs=n_epochs, scorers={'environment': evaluate_scorer})):
    if metrics.get("environment") > currentMax:
        currentMax = metrics.get("environment")
        dqn_max.copy_q_function_from(dqn)
    else:
        dqn.copy_q_function_from(dqn_max)
    
    dqn.save_model(model_file)
    if currentMax > 100: # For the purpose of the exercise the training will stop if the agent manages to complete the level
        print("A suitable model has been found.")
        break

print("Done!")


### 2.4 Validation
Now let's see if the training did something. If the results are not as expected try recording more data or increasing the training epochs.

In [ ]:
env_show = Env(visible=True, level=level).env
dqn = d3rlpy.algos.DQN()
dqn.build_with_dataset(getDataset())
dqn.load_model(model_file)

try:
    while True:
        observation = env_show.reset()
        done = False
        total_reward = 0
        while not done:
            observation, reward, done, info = env_show.step(
                dqn.predict([observation])[0])
            total_reward += reward

        print(f'finished episode, total_reward: {total_reward}')
except ConnectionResetError:
    print("Window closed.")


## 3. Offline RL vs Online RL 

Now we want to compare the approach from the first exercise where an online Q_Learner was used with the results we were able to achieve with the offline RL approach demonstrated above.

![Gif](https://1.bp.blogspot.com/-O0FvK3zJd9w/XpXqiJduwyI/AAAAAAAAFtM/5hxzdWOoSLw5sd5vEgMsiGVJSATKx1oEgCLcBGAsYHQ/s640/OFFLINE%2BRL%2Bfig1%2B05b.gif)


### 3.1 Reproducibility
To compare as fairly as possible we ran both the Online Q_Learner as well as the offline Deep Q Network until a plateau of performance has been reached.
- The online Q_Learner was able to train for 10.000 episodes per level while being able to interact with the environment
- The DQN was fed <1 hour (~360 episodes) of human playtime per level and was not allowed to interact with the environment while training
- Both models were using the same reward settings


### 3.2 Easy level

|  | Reward | Video |
| -------- | -------- | -------- |
| Q_Learner  | 232     | ![Gif](https://raw.githubusercontent.com/koerners/marioai/master/exercise_offline_rl/data/jupyter/rough_terrain_q_learner_232.gif)    |
| Deep Q Network  | 280    | ![Gif](https://raw.githubusercontent.com/koerners/marioai/master/exercise_offline_rl/data/jupyter/rough_terrain_dqn_280.gif)    |

With sufficient training, neither model struggles with the easy level. However, the model fed with player generated data shows better anticipation of jumps which leads to a better overall result.

### 3.3 Medium level

|  | Reward | Video |
| -------- | -------- | -------- |
| Q_Learner  | 176     | ![Gif](https://raw.githubusercontent.com/koerners/marioai/master/exercise_offline_rl/data/jupyter/cliff_and_enemies_q_learner_176.gif)    |
| Deep Q Network  | 193    | ![Gif](https://raw.githubusercontent.com/koerners/marioai/master/exercise_offline_rl/data/jupyter/cliff_and_enemies_dqn_193.gif)    |

In the medium level, both models behave similarly to the easy level. While the Online Learner seems to have a better strategy to avoid enemies, the Offline Learner has the better jumping performance leading to a better overall score as it is quicker to finish the level.

### 3.4 Hard level
|  | Reward | Video |
| -------- | -------- | -------- |
| Q_Learner  | -559     | ![Gif](https://raw.githubusercontent.com/koerners/marioai/master/exercise_offline_rl/data/jupyter/climb_q_learner_-559.gif)    |
| Deep Q Network  | -34    | ![Gif](https://raw.githubusercontent.com/koerners/marioai/master/exercise_offline_rl/data/jupyter/climb_dqn_-34.gif)    |

Neither model manages to complete the really hard level we tested them on. It is however interesting to observe the different strategies they applied. While the Q_Learner shows a very promising leap to the middle platform, failing to reach the final platform it seems to just give up and wait for the time to run out leading to a high time punishment and therefore an extremely low score. The offline trained model while still failing to complete the level, has developed a strategy to avoid the high time punishment by committing suicide as soon as possible.

### 3.5 Reward Summary

![Summary](https://raw.githubusercontent.com/koerners/marioai/master/exercise_offline_rl/data/jupyter/level-summary.png)




### 3.6 Performance over 100 level

// TODO -> With video maybe

## 4. Conclusion

While achieving slightly better results than the online RL approach in the easy and medium level, the offline DQN approach was not able to perform any better than the online implementation in the hard level we tested them both on. Our guess would be the lack of training data as mastering a level of this complexity would require a level of understanding of the environment that the network was not able to abstract from the data we were able to provide. This also demonstrates the biggest issue with the offline RL approach: it's high dependence on good and plentiful data. 
Creating such data is a time-consuming and often times expensive endeavor. 

In the real world offline RL is often times the only sensible way to use an ML model as online learning would simply be too dangerous or slow. 
However, this comes with the price of the agent not being able to explore different approaches on its own meaning an often times subpar training result.
Companies like Tesla have acknowledged this issue and have been training their self-driving cars in a life like simulated environment in which they reproduce difficult situations the car might encounter to explore without any real world damages. [[3]](https://youtu.be/11QXiJ8ORe8?t=3187)




// TODO: more stuff



![Thanks](https://media4.giphy.com/media/1mssFONYwmBlJy1DAv/giphy.gif?cid=ecf05e47fq7b3e8nbn49rxb2hj1f8qy627umny603h7tsi8f&rid=giphy.gif&ct=g)